In [6]:
########################################################
# STEP 1 - Merge all .csv files downloaded from Stack 
#          exchange data explorer
########################################################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from gensim.models.doc2vec import Doc2Vec

csv_file_list = ["QueryResults.csv"]

list_of_dataframes = []
for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))

merged_df = pd.concat(list_of_dataframes)
merged_df.to_csv('StackOverflowBDAQuestions.csv')

In [7]:
########################################################
# STEP 2 - Split central dataframe into seperate ones 
#          for questions and answers 
########################################################

df = pd.read_csv('StackOverflowBDAQuestions.csv')

questionsDF = df[['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'ClosedDate']].copy()
questionsDF = questionsDF.drop_duplicates()

answersDF = df[['Id.1', 'PostTypeId.1', 'ParentId', 'CreationDate.1', 'Score.1', 'ViewCount', 'Body.1', 'LastEditDate.1', 'LastActivityDate.1', 'CommentCount']].copy()
answersDF.columns = ['Id', 'PostTypeId', 'ParentId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'LastEditDate', 'LastActivityDate', 'CommentCount']

questionsDF.to_csv('StackOverflowBDAQuestions_Questions.csv')
answersDF.to_csv('StackOverflowBDAQuestions_Answers.csv')

In [10]:
########################################################
# STEP 3 - Tally import lines for questions and answers
########################################################

def appendLibrary(lan, lib, row):
    templist = [lan] + [lib] + questionsDF.iloc[row].values.tolist()
    LibraryDF.loc[len(LibraryDF)] = templist



filename = [
    'StackOverflowBDAQuestions_Questions.csv',
    'StackOverflowBDAQuestions_Answers.csv'
]

columnNames = [
    ['Language', 'Library', 'Unnamed: 0', 'Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'ClosedDate'],
    ['Language', 'Library', 'Unnamed: 0', 'Id', 'PostTypeId', 'ParentId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'LastEditDate', 'LastActivityDate', 'CommentCount']
]



for i in range(2):
    # Import questions into dataframe
    questionsDF = pd.read_csv(filename[i])

    # Create dataframe
    LibraryDF = pd.DataFrame(columns=columnNames[i])



    # C/C++
    for index, element in questionsDF.iterrows():
        text = element['Body']
        pattern = "#include &lt;(.*?)&gt;"

        for m in re.finditer(pattern, text):
            appendLibrary("C/C++", m.group(1), index)

    for index, element in questionsDF.iterrows():
        text = element['Body']
        pattern = '#include "(.*?)"'

        for m in re.finditer(pattern, text):
            appendLibrary("C/C++", m.group(1), index)



    # Java    -     import library;
    for index, element in questionsDF.iterrows():
        text = element['Body']
        pattern = "import (.*?);"

        for m in re.finditer(pattern, text):
            text = m.group(1)
            
            # Clean java mistakes
            if ' ' in str(text):
                if ' from ' in str(text):
                    pattern = "(?<=from ).*"
                    for m in re.finditer(pattern, text):
                        text = m.group()

            if ((text[0]=='"') and (text[-1]=='"')) or ((text[0]=="'") and (text[-1]=="'")):
                text = text[1:-1]

            if '&quot' in text:                     # ending in &quot
                pattern = '(?:(?!&quot).)*'
                for m in re.finditer(pattern, text):
                        text = m.group()
                        break

            if ' ' in text:                         # keep after "static "
                pattern = "(?<=static ).*"
                for m in re.finditer(pattern, text):
                    text = m.group()

            if ' ' in text:                         # up to first ' '
                pattern = '([^\s]+)'
                for m in re.finditer(pattern, text):
                    text = m.group()
                    break
            
            appendLibrary("Java", text, index)



    # Python    -     import library   or   from library import specific    or    import library as pseudo
    for index, element in questionsDF.iterrows():
        text = element['Body']

        pattern = "import (.*?)\n"
        for m in re.finditer(pattern, text):
            if (';' not in m.group(1)) and \
            ('*' not in m.group(1)) and \
            ('<' not in m.group(1)) and \
            ('>' not in m.group(1)) and \
            ('{' not in m.group(1)) and \
            ('}' not in m.group(1)) and \
            ('(' not in m.group(1)) and \
            (')' not in m.group(1)):
                result = m.group(1).split(' ', 1)[0]
                result = result.split(',', 1)[0]
                result = result.split('.', 1)[0]
                appendLibrary("Python", result, index)

        pattern = "import (.*?) "
        for m in re.finditer(pattern, text):
            if (';' not in m.group(1)) and \
            ('*' not in m.group(1)) and \
            ('<' not in m.group(1)) and \
            ('>' not in m.group(1)) and \
            ('{' not in m.group(1)) and \
            ('}' not in m.group(1)) and \
            ('(' not in m.group(1)) and \
            (')' not in m.group(1)):
                result = m.group(1).split(' ', 1)[0]
                result = result.split(',', 1)[0]
                result = result.split('.', 1)[0]
                appendLibrary("Python", result, index)

        pattern = "from (.*?) import"
        for m in re.finditer(pattern, text):
            if (';' not in m.group(1)) and \
            ('*' not in m.group(1)) and \
            ('<' not in m.group(1)) and \
            ('>' not in m.group(1)) and \
            ('{' not in m.group(1)) and \
            ('}' not in m.group(1)) and \
            ('(' not in m.group(1)) and \
            (')' not in m.group(1)):
                result = m.group(1).split(' ', 1)[0]
                result = result.split(',', 1)[0]
                result = result.split('.', 1)[0]
                appendLibrary("Python", result, index)



    # Javascript    -     <script src="libraryURL">
    for index, element in questionsDF.iterrows():
        text = element['Body']
        pattern = "&lt;script src=&quot;(.*?)&quot;&gt;"

        for m in re.finditer(pattern, text):
            text = m.group(1)
            if "@" in text:
                text = text.split('@', 1)[1]
            if "gstatic.com/" in text:
                text = text.split('.com/', 1)[1]
            if (text[0] != '/') and ('https' not in text):
                text = text.split('/', 1)[0]
            if 'firebase' in text:
                text = 'firebase'
            if 'materialize.min.js' in text:
                text = 'materialize'
            if 'http' in text:
                text = text.split('/')[-1]
            if ('.js' in text) or ('&quot;' in text):
                continue

            appendLibrary("JavaScript", text, index)



    # R    -     library(library)
    for index, element in questionsDF.iterrows():
        text = element['Body']
        pattern = "library\((.*?)\)"

        for m in re.finditer(pattern, text):
            text = m.group(1)

            if text:
                if text[0] != ' ':
                    if ' ' in text:
                        text = text.split(' ', 1)[0]
                else:
                    text = text[1:]
                    if ' ' in text:
                        text = text.split(' ', 1)[0]

                if '&quot;' in text:
                    text = text.split('&quot;', 2)[1]

                if (text[0] == "'") or (text[0] == '"'):
                    text = text[1:-1]

                appendLibrary("R", text, index)



    # Ruby        rb              require 'library'
    for index, element in questionsDF.iterrows():
        text = element['Body']
        pattern = "require '(.*?)'"

        for m in re.finditer(pattern, text):
            text = m.group(1)
            if text:    
                if (text[0] == '.') and (text[1] == '/'):
                    text = text[2:]
                if ' ' in text:
                    text = text.split(' ', 1)[0]
                if '/' in text:
                    text = text.split('/', 1)[0]

                appendLibrary("Ruby", text, index)

        text = element['Body']
        pattern = 'require "(.*?)"'
        for m in re.finditer(pattern, text):
            text = m.group(1)
            if text:
                if (len(text) > 25):
                    continue
                if ' ' in text:
                    text = text.split(' ', 1)[0]
                if '/' in text:
                    text = text.split('/', 1)[0]

                appendLibrary("Ruby", text, index)



    LibraryDF = LibraryDF[LibraryDF.Library != "&quot"]
    LibraryDF = LibraryDF[LibraryDF.Library != ""]
    
    
    mod = Doc2Vec.load ('doc2vecR.200.30.20.5.1550908281.eAp.trained')

                
    # Reformat numeric and datetime columns correctly 
    if i==0:
        LibraryDF["Unnamed: 0"] = pd.to_numeric(LibraryDF["Unnamed: 0"])
        LibraryDF["Id"] = pd.to_numeric(LibraryDF["Id"])
        LibraryDF["PostTypeId"] = pd.to_numeric(LibraryDF["PostTypeId"])
        LibraryDF["AcceptedAnswerId"] = pd.to_numeric(LibraryDF["AcceptedAnswerId"])
        LibraryDF["CreationDate"] = pd.to_datetime(LibraryDF["CreationDate"])
        LibraryDF["Score"] = pd.to_numeric(LibraryDF["Score"])
        LibraryDF["ViewCount"] = pd.to_numeric(LibraryDF["ViewCount"])
        LibraryDF["LastEditDate"] = pd.to_datetime(LibraryDF["LastEditDate"])
        LibraryDF["LastActivityDate"] = pd.to_datetime(LibraryDF["LastActivityDate"])
        LibraryDF["AnswerCount"] = pd.to_numeric(LibraryDF["AnswerCount"])
        LibraryDF["ClosedDate"] = pd.to_datetime(LibraryDF["ClosedDate"])
        
        LibraryDF.to_csv('SO_BDA_Q_Libraries.csv')
        
        # Remove all rows that have libraries not in the Skill Space 
        LibraryDF = LibraryDF[LibraryDF['Library'].isin(mod.wv.vocab)]

        LibraryDF.to_csv('SO_BDA_Q_Libraries_in_Skill_Space.csv')

    else:
        LibraryDF["Unnamed: 0"] = pd.to_numeric(LibraryDF["Unnamed: 0"])
        LibraryDF["Id"] = pd.to_numeric(LibraryDF["Id"])
        LibraryDF["PostTypeId"] = pd.to_numeric(LibraryDF["PostTypeId"])
        LibraryDF["ParentId"] = pd.to_numeric(LibraryDF["ParentId"])
        LibraryDF["CreationDate"] = pd.to_datetime(LibraryDF["CreationDate"])
        LibraryDF["Score"] = pd.to_numeric(LibraryDF["Score"])
        LibraryDF["ViewCount"] = pd.to_numeric(LibraryDF["ViewCount"])
        LibraryDF["LastEditDate"] = pd.to_datetime(LibraryDF["LastEditDate"])
        LibraryDF["LastActivityDate"] = pd.to_datetime(LibraryDF["LastActivityDate"])
        LibraryDF["CommentCount"] = pd.to_numeric(LibraryDF["CommentCount"])

        LibraryDF.to_csv('SO_BDA_A_Libraries.csv')
        
        # Remove all rows that have libraries not in the Skill Space 
        LibraryDF = LibraryDF[LibraryDF['Library'].isin(mod.wv.vocab)]        
        
        LibraryDF.to_csv('SO_BDA_A_Libraries_in_Skill_Space.csv')